In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random 
import time
from scipy.stats import rankdata

In [2]:
def TDT(n):
    b = n[0] + n[2] + 2*n[3]
    c = n[1] + n[2] + 2*n[4]
    
    if b == 0 and c == 0:
        return 0
    elif b == 0:
        return 1000000000
    else:
        return (b-c)**2/(b+c)

In [3]:
def lap_normal(stats, s, epsilon, K, m):
    g = sorted(stats, reverse=True)
    pri_data = g + np.random.laplace(loc = 0.0, scale = (2*K*s)/epsilon, size=np.size(stats))
    pri_r = np.argsort(pri_data)
    
    return pri_r[m-K:m]

In [4]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_stats(stats, s, epsilon, K, m):
    g = sorted(stats, reverse=True)
    
    h = np.zeros(m)
    for i in range(m):
        h[i] = g[i]
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * h[i] / (2*K*s))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        h[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [5]:
def SHD(cc, n):
    T = TDT(n)
    
    d = np.zeros(2)
    N = np.zeros(6)
    
    for k in range(6):
        N[k] = n[k]
    
    if T < cc:
        while T < cc:
            if N[4] > 0:
                N[4] -= 1
            elif N[1] > 0:
                N[1] -= 1
            elif N[2] > 0:
                N[2] -= 1
            elif N[5] > 0:
                N[5] -= 1
            else:
                N[0] -= 1
            N[3] += 1
            
            T = TDT(N)
            d[0] -= 1
        
        T = TDT(n)
        for k in range(6):
            N[k] = n[k]
        
        while T < cc:
            if N[3] > 0:
                N[3] -= 1
            elif N[0] > 0:
                N[0] -= 1
            elif N[2] > 0:
                N[2] -= 1
            elif N[5] > 0:
                N[5] -= 1
            else:
                N[1] -= 1
            N[4] += 1
            
            T = TDT(N)
            d[1] -= 1
        
        if d[0] > d[1]:
            return d[0]
        else:
            return d[1]
    
    else:
        step = 0
        
        if n[0] + 2*n[3] > n[1] + 2*n[4]:
            while T >= cc:
                if n[3] > 0:
                    n[3] -= 1
                elif n[0] > 0:
                    n[0] -= 1
                elif n[5] > 0:
                    n[5] -= 1
                elif n[2] > 0:
                    n[2] -= 1
                else:
                    n[1] -= 1
                n[4] += 1
            
                T = TDT(n)
                step += 1
        else:
            while T >= cc:
                if n[4] > 0:
                    n[4] -= 1
                elif n[1] > 0:
                    n[1] -= 1
                elif n[5] > 0:
                    n[5] -= 1
                elif n[2] > 0:
                    n[2] -= 1
                else:
                    n[0] -= 1
                n[3] += 1
            
                T = TDT(n)
                step += 1
        
        return step-1

def exp_shd(n, epsilon, K, m):
    shd = np.zeros(m)
    cc = 19.5
    
    for i in range(m):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        shd[i] = SHD(cc,k)
    
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * shd[i] / (2*K))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        shd[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [6]:
def FPA(stats, sensitivity, epsilon, K, m):
    rank = np.argsort(stats)
    g = np.zeros(m)
    for i in range(m):
        g[i] = stats[i]
        
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    pri_x = np.zeros(m)
    
    s = int(m/(2*K))
    
    l = 2*K*sensitivity*np.sqrt(s)/epsilon
    
    prime_y[0:s] = y[0:s] + np.random.laplace(loc= 0.0, scale = l, size = s)
    
    pri_x = np.fft.ifft(prime_y).real
    
    pri_r = np.argsort(pri_x)
    
    #for i in range(K):
    #    for j in range(m):
    #        if pri_r[m-i-1] == rank[m-j-1]:
    #            pri_r[m-i-1] = j
    #            break
    
    return pri_r[m-K:m]

In [7]:
def extended_FPA(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(M)-i] = prime_y.real[i]
            prime_y.imag[int(M)-i] = -prime_y.imag[i]
    
    #prime_g = np.fft.ifft(prime_y)
    
    l = 2*K*sensitivity/epsilon
    
    pri_x = np.zeros(m)
    
    noise = np.zeros(m)
    
    for j in range(m):
        hat_y = np.fft.fft(zeros)
        p = np.random.laplace(loc= 0.0, scale = l, size = 1)
        hat_y.real[0] = prime_y.real[0] + p
        for k in range(1,s):
            hat_y.real[k] = prime_y.real[k] + p * math.cos(-2*j*k*math.pi/m)
            hat_y.imag[k] = prime_y.imag[k] + p * math.sin(-2*j*k*math.pi/m)
            hat_y.real[m-k] = hat_y.real[k]
            hat_y.imag[m-k] = -hat_y.imag[k]
            
        pri_x[j] = np.fft.ifft(hat_y).real[j]
        
    pri_r = np.argsort(pri_x)
    
    return pri_r[m-K:m]

In [8]:
def lap_DFT(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(M)-i] = prime_y.real[i]
            prime_y.imag[int(M)-i] = -prime_y.imag[i]
    
    prime_g = np.fft.ifft(prime_y)
    
    pri_g = prime_g.real + np.random.laplace(loc = 0.0, scale = (2*K)*((2*s-1)/m)*sensitivity/epsilon, size=m)
    
    pri_r = np.argsort(pri_g)
    
    return pri_r[m-K:m]

In [9]:
def exp_DFT(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(m)-i] = prime_y.real[i]
            prime_y.imag[int(m)-i] = -prime_y.imag[i]
    
    prime_g = np.fft.ifft(prime_y).real
    
    h = np.zeros(m)
    for i in range(m):
        h[i] = prime_g[i]
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp((epsilon * h[i] / (2*K*sensitivity)) * (m / (2*s-1)))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        h[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [10]:
def runTime(K, N, M):
    S = np.zeros(M)
    b = np.zeros(M)
    c = np.zeros(M)
    
    n = np.zeros((M,6))
    
    sig = random.sample(range(M),10)
    
    for i in range(10):
        S[sig[i]] = random.randint(0,2*N)
        b[sig[i]] = np.random.binomial(S[sig[i]],0.75)
        c[sig[i]] = S[sig[i]] - b[sig[i]]
        n[sig[i]][0] = b[sig[i]]
        n[sig[i]][1] = c[sig[i]]
        n[sig[i]][5] = 2*N - b[sig[i]] - c[sig[i]]
    
    for i in range(M):
        if S[i] == 0:
            S[i] = random.randint(0,2*N)
            b[i] = np.random.binomial(S[i],0.5)
            c[i] = S[i] - b[i]
            n[i][0] = b[i]
            n[i][1] = c[i]
            n[i][5] = 2*N - b[i] - c[i]
        
    stats = np.zeros(M)
    order = np.zeros(M)
    
    for i in range(M):
        order[i] = i
        if b[i] == 0 and c[i] == 0:
            stats[i] = 0
        else:
            stats[i] = (b[i]-c[i])**2/(b[i]+c[i])
    
    x = np.argsort(stats)
    #for i in range(10):
    #    print(x[M-1-i])
    
    nn = np.zeros((M,6))
    for i in range(M):
        nn[i] = n[x[M-1-i]]
    
    sensitivity = 8*(N-1)/N
        
    RT = np.zeros(7)
    
    start = time.time()
    lap_normal(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[0] = end-start
    
    start = time.time()
    exp_stats(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[1] = end-start
    
    start = time.time()
    exp_shd(nn, 3, K, M)
    end = time.time()
    RT[2] = end-start
    
    start = time.time()
    FPA(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[3] = end-start
    
    start = time.time()
    extended_FPA(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[4] = end-start
    
    start = time.time()
    lap_DFT(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[5] = end-start
    
    start = time.time()
    exp_DFT(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[6] = end-start
    
    return RT

In [11]:
#K = 1

N = 150
M = 5000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(1,150,5000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 0.0028569698333740234 [sec]
exp_stats_time = 0.009452104568481445 [sec]
exp_shd_time = 0.893402099609375 [sec]
FPA_time = 0.0016651153564453125 [sec]
extended_FPA_time = 71.24861598014832 [sec]
lap_DFT_time = 0.005319118499755859 [sec]
exp_DFT_time = 0.011432886123657227 [sec]
------------------
lap_normal_time = 0.0023202896118164062 [sec]
exp_stats_time = 0.006985902786254883 [sec]
exp_shd_time = 0.7913739681243896 [sec]
FPA_time = 0.0016460418701171875 [sec]
extended_FPA_time = 69.9875099658966 [sec]
lap_DFT_time = 0.005090951919555664 [sec]
exp_DFT_time = 0.011564970016479492 [sec]
------------------
lap_normal_time = 0.0022978782653808594 [sec]
exp_stats_time = 0.007002115249633789 [sec]
exp_shd_time = 0.7891719341278076 [sec]
FPA_time = 0.0016789436340332031 [sec]
extended_FPA_time = 69.92386722564697 [sec]
lap_DFT_time = 0.005131959915161133 [sec]
exp_DFT_time = 0.01139211654663086 [sec]
------------------
lap_normal_time = 0.002131938934326172 [sec]
exp_stats_

In [12]:
#K = 3

N = 150
M = 5000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(3,150,5000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 0.0024688243865966797 [sec]
exp_stats_time = 0.02020096778869629 [sec]
exp_shd_time = 0.798084020614624 [sec]
FPA_time = 0.0013418197631835938 [sec]
extended_FPA_time = 23.21733808517456 [sec]
lap_DFT_time = 0.003349781036376953 [sec]
exp_DFT_time = 0.023017406463623047 [sec]
------------------
lap_normal_time = 0.0021729469299316406 [sec]
exp_stats_time = 0.018012046813964844 [sec]
exp_shd_time = 0.8149271011352539 [sec]
FPA_time = 0.0013439655303955078 [sec]
extended_FPA_time = 23.263381958007812 [sec]
lap_DFT_time = 0.003329038619995117 [sec]
exp_DFT_time = 0.021699190139770508 [sec]
------------------
lap_normal_time = 0.0021409988403320312 [sec]
exp_stats_time = 0.018486976623535156 [sec]
exp_shd_time = 0.8251290321350098 [sec]
FPA_time = 0.0013539791107177734 [sec]
extended_FPA_time = 23.538076877593994 [sec]
lap_DFT_time = 0.0031621456146240234 [sec]
exp_DFT_time = 0.02195000648498535 [sec]
------------------
lap_normal_time = 0.002187967300415039 [sec]
exp_sta

In [13]:
#K = 5

N = 150
M = 5000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(5,150,5000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 0.002377033233642578 [sec]
exp_stats_time = 0.029613018035888672 [sec]
exp_shd_time = 0.8263349533081055 [sec]
FPA_time = 0.0014576911926269531 [sec]
extended_FPA_time = 14.252799034118652 [sec]
lap_DFT_time = 0.0027587413787841797 [sec]
exp_DFT_time = 0.033869028091430664 [sec]
------------------
lap_normal_time = 0.0022242069244384766 [sec]
exp_stats_time = 0.02850198745727539 [sec]
exp_shd_time = 0.8083739280700684 [sec]
FPA_time = 0.0013301372528076172 [sec]
extended_FPA_time = 14.216310739517212 [sec]
lap_DFT_time = 0.002869844436645508 [sec]
exp_DFT_time = 0.0363461971282959 [sec]
------------------
lap_normal_time = 0.0023097991943359375 [sec]
exp_stats_time = 0.028382062911987305 [sec]
exp_shd_time = 0.8139729499816895 [sec]
FPA_time = 0.0013499259948730469 [sec]
extended_FPA_time = 14.319423913955688 [sec]
lap_DFT_time = 0.0027611255645751953 [sec]
exp_DFT_time = 0.03432011604309082 [sec]
------------------
lap_normal_time = 0.0023360252380371094 [sec]
exp_st

In [14]:
#K = 10

N = 150
M = 5000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(10,150,5000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 0.002335071563720703 [sec]
exp_stats_time = 0.055558204650878906 [sec]
exp_shd_time = 0.824199914932251 [sec]
FPA_time = 0.001322031021118164 [sec]
extended_FPA_time = 7.382357120513916 [sec]
lap_DFT_time = 0.002549886703491211 [sec]
exp_DFT_time = 0.06447196006774902 [sec]
------------------
lap_normal_time = 0.0021440982818603516 [sec]
exp_stats_time = 0.05434703826904297 [sec]
exp_shd_time = 0.8375539779663086 [sec]
FPA_time = 0.001348733901977539 [sec]
extended_FPA_time = 7.419478178024292 [sec]
lap_DFT_time = 0.0027000904083251953 [sec]
exp_DFT_time = 0.06623983383178711 [sec]
------------------
lap_normal_time = 0.002226114273071289 [sec]
exp_stats_time = 0.05934286117553711 [sec]
exp_shd_time = 0.8807179927825928 [sec]
FPA_time = 0.0013918876647949219 [sec]
extended_FPA_time = 7.434281826019287 [sec]
lap_DFT_time = 0.0026328563690185547 [sec]
exp_DFT_time = 0.06397104263305664 [sec]
------------------
lap_normal_time = 0.002218961715698242 [sec]
exp_stats_time 